In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm

from scipy import stats
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

# =============================================================================
# 0. 데이터 로드 & 월별 집계 (value, weight)
# =============================================================================

print("="*80)
print("데이터 로드 및 월별 집계")
print("="*80)

train = pd.read_csv("./data/train.csv")

monthly = (
    train
    .groupby(["item_id", "year", "month"], as_index=False)
    .agg({'value': 'sum', 'weight': 'sum'})
)

monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

pivot = monthly.pivot(index="item_id", columns="ym", values="value").fillna(0.0)
pivot_weight = monthly.pivot(index="item_id", columns="ym", values="weight").fillna(0.0)
months_dt = pivot.columns.to_list()

print(f"데이터 로드 완료: {len(pivot)}개 품목, {len(months_dt)}개월")


# =============================================================================
# 1. Corr + DTW 기반 comovement pair 탐색 (pairs_with_dtw)
# =============================================================================

def safe_corr_with_pvalue(x, y):
    if np.std(x) == 0 or np.std(y) == 0:
        return 0.0, 1.0
    corr, p_value = pearsonr(x, y)
    return float(corr), float(p_value)


def calculate_score_advanced(corr, p_value, lag):
    """
    상관계수 + p-value + lag 기반 base score (0~100 정도 스케일)
    """
    # 상관계수 (0~70점)
    corr_score = abs(corr) * 70

    # 유의성 점수
    if p_value < 0.001:
        sig_score = 20
    elif p_value < 0.01:
        sig_score = 15
    elif p_value < 0.05:
        sig_score = 10
    else:
        sig_score = 5

    # Lag 점수
    if lag <= 2:
        lag_score = 10
    elif lag <= 4:
        lag_score = 7
    elif lag <= 6:
        lag_score = 4
    else:
        lag_score = 1

    return corr_score + sig_score + lag_score


def dtw_distance(series_A, series_B):
    """
    Dynamic Time Warping 거리 계산
    """
    n, m = len(series_A), len(series_B)
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = abs(series_A[i-1] - series_B[j-1])
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i-1, j],
                dtw_matrix[i, j-1],
                dtw_matrix[i-1, j-1]
            )

    return dtw_matrix[n, m]


def calculate_normalized_dtw(x, y, lag):
    """
    정규화 + DTW 거리 계산
    x: 선행 품목
    y: 후행 품목
    lag: 시차
    """
    if len(x) <= lag or len(y) <= lag:
        return float('inf')

    x_aligned = x[:-lag]
    y_aligned = y[lag:]

    if len(x_aligned) < 5:
        return float('inf')

    # 정규화
    scaler = StandardScaler()
    normalized_x = scaler.fit_transform(x_aligned.reshape(-1, 1)).flatten()
    normalized_y = scaler.fit_transform(y_aligned.reshape(-1, 1)).flatten()

    # DTW 거리
    distance = dtw_distance(normalized_x, normalized_y)

    # 길이로 정규화
    normalized_distance = distance / len(normalized_x)
    return normalized_distance


def find_comovement_pairs_with_dtw(
    pivot,
    max_lag=9,
    min_nonzero=12,
    corr_threshold=0.33,
    score_threshold=40,
    use_dtw=True,
    dtw_threshold=1.0
):
    """
    corr + p-value + lag + (선택적 DTW) 기반 comovement pair 탐색
    - 출력: pairs_with_dtw
      [leading_item_id, following_item_id, best_lag, max_corr, p_value, dtw_distance, comovement_score]
    """
    items = pivot.index.to_list()
    months = pivot.columns.to_list()
    n_months = len(months)

    results = []

    for i, leader in tqdm(enumerate(items), total=len(items), desc="Finding pairs (DTW)"):
        x = pivot.loc[leader].values.astype(float)
        if np.count_nonzero(x) < min_nonzero:
            continue

        candidates = []

        for follower in items:
            if follower == leader:
                continue

            y = pivot.loc[follower].values.astype(float)
            if np.count_nonzero(y) < min_nonzero:
                continue

            best_lag = None
            best_corr = 0.0
            best_p_value = 1.0
            best_dtw = float('inf')

            # lag 탐색
            for lag in range(1, max_lag + 1):
                if n_months <= lag:
                    continue

                corr, p_value = safe_corr_with_pvalue(x[:-lag], y[lag:])

                if abs(corr) > abs(best_corr):
                    best_corr = corr
                    best_lag = lag
                    best_p_value = p_value

            # 1단계: 상관계수 임계값 통과 시
            if best_lag is not None and abs(best_corr) >= corr_threshold:

                # 2단계: DTW 검증 (옵션)
                if use_dtw:
                    best_dtw = calculate_normalized_dtw(x, y, best_lag)

                    # DTW 임계값 통과 못하면 스킵
                    if best_dtw > dtw_threshold:
                        continue

                # 종합 점수 계산
                score = calculate_score_advanced(best_corr, best_p_value, best_lag)

                # DTW 점수 반영 (옵션)
                if use_dtw and best_dtw != float('inf'):
                    # DTW가 낮을수록 좋으므로 역수 사용
                    dtw_bonus = 1 / (1 + best_dtw)
                    score = score * (1 + dtw_bonus)  # DTW 보너스 추가

                # 점수 임계값 통과 시 후보에 추가
                if score >= score_threshold:
                    candidates.append({
                        "following_item_id": follower,
                        "best_lag": best_lag,
                        "max_corr": best_corr,
                        "p_value": best_p_value,
                        "dtw_distance": best_dtw if use_dtw else None,
                        "comovement_score": score
                    })

        # 점수 순으로 정렬 (높은 순)
        candidates.sort(key=lambda x: -x['comovement_score'])

        for candidate in candidates:
            result_dict = {
                "leading_item_id": leader,
                "following_item_id": candidate["following_item_id"],
                "best_lag": candidate["best_lag"],
                "max_corr": candidate["max_corr"],
                "p_value": candidate["p_value"],
                "dtw_distance": candidate["dtw_distance"],
                "comovement_score": candidate["comovement_score"],
            }

            results.append(result_dict)

    pairs = pd.DataFrame(results)
    print(f"\nPairs with DTW: {len(pairs)}개")
    print(pairs.head())
    return pairs


# =============================================================================
# 2. 2,700개 목표 노이즈 제거 
# =============================================================================

def calculate_quality_score_moderate(a_series, b_series, lag):
    """
    중간 품질 점수 계산 (0-100점)
    - 원본 3,000대 → 목표 2,700개
    - 적절한 노이즈 제거 (80% 유지)
    """
    scores = {}

    # 1. 0 비율 (10점) - 중간
    zero_ratio_a = (a_series == 0).sum() / len(a_series)
    zero_ratio_b = (b_series == 0).sum() / len(b_series)
    max_zero = max(zero_ratio_a, zero_ratio_b)

    if max_zero < 0.17:
        scores['zero'] = 10
    elif max_zero < 0.37:
        scores['zero'] = 7
    elif max_zero < 0.57:  # 57%까지 허용
        scores['zero'] = 4
    else:
        scores['zero'] = 0

    # 2. 변동계수 (10점) - 중간
    cv_a = np.std(a_series) / (np.mean(a_series) + 1)
    cv_b = np.std(b_series) / (np.mean(b_series) + 1)
    max_cv = max(cv_a, cv_b)

    if max_cv < 0.75:
        scores['cv'] = 10
    elif max_cv < 1.4:
        scores['cv'] = 7
    elif max_cv < 2.7:  # 2.7까지 허용
        scores['cv'] = 4
    else:
        scores['cv'] = 0

    # 3. 스파이크 (10점) - 중간
    a_changes = np.abs(np.diff(a_series))
    b_changes = np.abs(np.diff(b_series))

    if len(a_changes) > 0 and len(b_changes) > 0:
        a_spike = a_changes.max() / (np.median(a_changes) + 1)
        b_spike = b_changes.max() / (np.median(b_changes) + 1)
        max_spike = max(a_spike, b_spike)

        if max_spike < 4.5:
            scores['spike'] = 10
        elif max_spike < 9.5:
            scores['spike'] = 7
        elif max_spike < 17:  # 17까지 허용
            scores['spike'] = 4
        else:
            scores['spike'] = 0
    else:
        scores['spike'] = 5

    # 4. 안정성 (15점) - 중간
    n = len(a_series)

    try:
        mid = n // 2

        if mid >= 12 and mid + lag < len(b_series):
            corr_first = np.corrcoef(
                a_series[:mid],
                b_series[lag:mid+lag]
            )[0, 1]

            corr_second = np.corrcoef(
                a_series[mid:],
                b_series[mid+lag:]
            )[0, 1]

            if not np.isnan(corr_first) and not np.isnan(corr_second):
                consistency = 1 - abs(corr_first - corr_second)

                if consistency > 0.52:  # 중간
                    scores['stability'] = 15
                elif consistency > 0.35:
                    scores['stability'] = 10
                elif consistency > 0.22:
                    scores['stability'] = 6
                else:
                    scores['stability'] = 3
            else:
                scores['stability'] = 5
        else:
            scores['stability'] = 5
    except:
        scores['stability'] = 5

    # 5. 트렌드 일치 (10점) - 중간
    if len(a_series) > 0 and len(b_series) > 0:
        x = np.arange(len(a_series))

        if np.std(a_series) > 0:
            slope_a, _ = np.polyfit(x, a_series, 1)
        else:
            slope_a = 0

        if np.std(b_series) > 0:
            slope_b, _ = np.polyfit(x, b_series, 1)
        else:
            slope_b = 0

        if (slope_a > 0 and slope_b > 0) or (slope_a < 0 and slope_b < 0):
            ratio = abs(slope_a) / (abs(slope_b) + 1e-10)
            if 0.38 < ratio < 2.7:  # 중간
                scores['trend'] = 10
            elif 0.25 < ratio < 4.2:
                scores['trend'] = 6
            else:
                scores['trend'] = 3
        elif slope_a == 0 or slope_b == 0:
            scores['trend'] = 5
        else:
            scores['trend'] = 2
    else:
        scores['trend'] = 4

    # 6. 방향 일치도 (10점) - 중간
    if len(a_changes) > lag and len(b_changes) > lag:
        direction_match = (np.sign(a_changes[:-lag]) == np.sign(b_changes[lag:])).mean()

        if direction_match > 0.64:  # 중간
            scores['direction'] = 10
        elif direction_match > 0.54:
            scores['direction'] = 7
        elif direction_match > 0.44:
            scores['direction'] = 4
        else:
            scores['direction'] = 1
    else:
        scores['direction'] = 4

    # 7. 통계적 유의성 (10점) - 중간
    try:
        if len(a_series) > lag:
            _, p_value = stats.pearsonr(
                a_series[:-lag] if lag > 0 else a_series,
                b_series[lag:] if lag > 0 else b_series
            )

            if p_value < 0.007:  # 중간
                scores['significance'] = 10
            elif p_value < 0.025:
                scores['significance'] = 7
            elif p_value < 0.07:
                scores['significance'] = 4
            else:
                scores['significance'] = 1
        else:
            scores['significance'] = 4
    except:
        scores['significance'] = 4

    # 8. 자기상관 (10점) - 중간
    try:
        if len(b_series) > 1:
            autocorr = np.corrcoef(b_series[:-1], b_series[1:])[0, 1]

            if 0.24 < autocorr < 0.86:  # 중간
                scores['autocorr'] = 10
            elif 0.09 < autocorr < 0.91:
                scores['autocorr'] = 7
            else:
                scores['autocorr'] = 3
        else:
            scores['autocorr'] = 4
    except:
        scores['autocorr'] = 4

    # 9. 이상치 비율 (10점) - 중간
    def count_outliers(series):
        q1 = np.percentile(series, 25)
        q3 = np.percentile(series, 75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        outliers = ((series < lower) | (series > upper)).sum()
        return outliers / len(series)

    outlier_ratio_a = count_outliers(a_series)
    outlier_ratio_b = count_outliers(b_series)
    max_outlier = max(outlier_ratio_a, outlier_ratio_b)

    if max_outlier < 0.085:
        scores['outlier'] = 10
    elif max_outlier < 0.16:
        scores['outlier'] = 7
    elif max_outlier < 0.26:  # 26%까지 허용
        scores['outlier'] = 4
    else:
        scores['outlier'] = 1

    # 10. 분포 정규성 (5점) - 중간
    try:
        if len(b_series) >= 8:
            _, p_shapiro = stats.shapiro(b_series)

            if p_shapiro > 0.02:  # 중간
                scores['normality'] = 5
            elif p_shapiro > 0.004:
                scores['normality'] = 3
            else:
                scores['normality'] = 1
        else:
            scores['normality'] = 2
    except:
        scores['normality'] = 2

    total_score = sum(scores.values())
    return total_score, scores


def remove_noise_pairs_for_2700(pairs, pivot, target_count=2700):
    """
    2,700개 목표 노이즈 제거 (중간 모드)
    """
    print("\n" + "="*80)
    print("2,700개 목표 노이즈 제거 (중간 모드)")
    print("="*80)
    print(f"원본: {len(pairs):,}개")
    print(f"목표: {target_count:,}개 (약 {target_count/len(pairs)*100:.0f}% 유지)")

    quality_scores = []
    detailed_scores = []

    # 품질 점수 계산 (중간 기준)
    print(f"\n품질 평가 중...")
    for row in tqdm(pairs.itertuples(), total=len(pairs), desc="평가"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)

        if leader not in pivot.index or follower not in pivot.index:
            quality_scores.append(0)
            detailed_scores.append({})
            continue

        a_series = pivot.loc[leader].values
        b_series = pivot.loc[follower].values

        total_score, scores = calculate_quality_score_moderate(a_series, b_series, lag)

        quality_scores.append(total_score)
        detailed_scores.append(scores)

    # 품질 점수 추가
    pairs_with_score = pairs.copy()
    pairs_with_score['quality_score'] = quality_scores

    # 목표 개수 기반 임계값 계산
    sorted_scores = sorted(quality_scores, reverse=True)

    if len(sorted_scores) >= target_count:
        threshold = sorted_scores[target_count - 1]
        threshold = max(33, (threshold // 5) * 5)  # 최소 33점, 5점 단위 내림
    else:
        threshold = 33

    # 필터링
    pairs_clean = pairs_with_score[
        pairs_with_score['quality_score'] >= threshold
    ].copy()

    # 통계
    removed = len(pairs) - len(pairs_clean)
    removed_ratio = removed / len(pairs) * 100
    kept_ratio = len(pairs_clean) / len(pairs) * 100

    print(f"\n{'='*80}")
    print(f"노이즈 제거 완료!")
    print(f"{'='*80}")
    print(f"원본: {len(pairs):,}개")
    print(f"제거: {removed:,}개 ({removed_ratio:.1f}%)")
    print(f"유지: {len(pairs_clean):,}개 ({kept_ratio:.1f}%)")
    print(f"임계값: {threshold}점")

    # 점수 분포
    print(f"\n품질 점수 분포:")
    print(f"  - 평균: {np.mean(quality_scores):.1f}점")
    print(f"  - 중앙값: {np.median(quality_scores):.1f}점")
    print(f"  - 최소: {np.min(quality_scores):.1f}점")
    print(f"  - 최대: {np.max(quality_scores):.1f}점")

    selected_scores = pairs_clean['quality_score'].values
    print(f"\n선택된 쌍의 점수:")
    print(f"  - 평균: {np.mean(selected_scores):.1f}점")
    print(f"  - 최소: {np.min(selected_scores):.1f}점")

    print(f"\n점수 구간별 분포:")
    bins = [0, 30, 40, 50, 60, 70, 80, 90, 100]
    for i in range(len(bins)-1):
        total_count = sum((np.array(quality_scores) >= bins[i]) & (np.array(quality_scores) < bins[i+1]))
        selected_count = sum((selected_scores >= bins[i]) & (selected_scores < bins[i+1]))
        print(f"  {bins[i]:3d}-{bins[i+1]:3d}점: 전체 {total_count:4,}개, 선택 {selected_count:4,}개")

    print(f"\n기준별 평균 점수 (선택된 쌍):")
    criteria_names = {
        'zero': '0 비율',
        'cv': '변동계수',
        'spike': '스파이크',
        'stability': '안정성',
        'trend': '트렌드 일치',
        'direction': '방향 일치도',
        'significance': '통계적 유의성',
        'autocorr': '자기상관',
        'outlier': '이상치 비율',
        'normality': '정규성'
    }

    selected_indices = pairs_with_score[pairs_with_score['quality_score'] >= threshold].index
    selected_detailed = [detailed_scores[i] for i in selected_indices if i < len(detailed_scores)]

    for key, name in criteria_names.items():
        scores = [s.get(key, 0) for s in selected_detailed if s]
        if scores:
            if key == 'stability':
                max_score = 15
            elif key == 'normality':
                max_score = 5
            else:
                max_score = 10

            print(f"  {name:15s}: 평균 {np.mean(scores):5.1f}점 (최대 {max_score}점)")

    # 품질 점수 컬럼 제거해서 반환
    pairs_for_model = pairs_clean.drop(columns=['quality_score'], errors='ignore')
    return pairs_for_model


# =============================================================================
# 3. 조건부 Weight 피처
# =============================================================================

def create_conditional_weight_features(value, weight, value_series):
    """조건부 Weight 피처 4개"""
    value_positive = value_series[value_series > 0]
    if len(value_positive) > 0:
        value_median = np.median(value_positive)
        value_q75 = np.percentile(value_positive, 75)
    else:
        value_median = 0.0
        value_q75 = 0.0

    if value > value_median:
        weight_for_high_value = weight * 1.5
    else:
        weight_for_high_value = 0.0

    weight_value_ratio = weight * (value / (value_median + 1))
    exp_weighted = weight * (1 - np.exp(-value / (value_median + 1)))

    if value > value_q75:
        weight_importance = weight * 1.5
    elif value > value_median:
        weight_importance = weight * 1.0
    else:
        weight_importance = weight * 0.1

    return {
        'weight_for_high_value': weight_for_high_value,
        'weight_value_ratio': weight_value_ratio,
        'exp_weighted': exp_weighted,
        'weight_importance': weight_importance
    }


# =============================================================================
# 4. 학습 데이터 구축 (32개 피처, log1p target)
# =============================================================================

def build_training_data_optimized(pivot, pivot_weight, pairs, months_dt):
    """32개 피처 학습 데이터 생성 (log1p(target))"""
    months = months_dt
    n_months = len(months)
    rows = []

    for row in pairs.itertuples(index=False):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue
        if leader not in pivot_weight.index or follower not in pivot_weight.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)
        b_weight_series = pivot_weight.loc[follower].values.astype(float)

        # 최소 12개월 + lag 만큼 확보
        for t in range(max(lag, 12), n_months - 1):
            b_t = b_series[t]
            b_t_1 = b_series[t - 1]
            b_t_2 = b_series[t - 2]
            b_t_12 = b_series[t - 12]
            a_t_lag = a_series[t - lag]
            b_t_plus_1 = b_series[t + 1]
            b_weight_t = b_weight_series[t]
            target_month = months[t + 1].month

            # A(leader) 관련
            a_t_lag_diff = a_series[t - lag] - a_series[t - lag - 1]
            a_momentum = a_series[t - lag] / (a_series[t - lag - 3] + 1)
            a_roll_mean_3 = (a_series[t-lag] + a_series[t-lag-1] + a_series[t-lag-2]) / 3

            # B(follower) 자기회귀/변동
            b_diff = b_t - b_t_1
            b_pct_change = (b_t - b_t_1) / (b_t_1 + 1)
            b_yoy_growth = (b_t - b_t_12) / (b_t_12 + 1)
            b_roll_mean_3 = (b_series[t] + b_series[t-1] + b_series[t-2]) / 3
            b_roll_mean_6 = b_series[t-5:t+1].mean()
            b_roll_std_3 = np.std([b_series[t], b_series[t-1], b_series[t-2]])

            # A-B 관계
            ab_ratio = a_t_lag / (b_t + 1)
            ab_diff = a_t_lag - b_t
            corr_weighted_a = a_t_lag * corr

            # 추가 시계열 특징
            b_cv = b_roll_std_3 / (b_roll_mean_3 + 1)
            b_acceleration = (b_t - b_t_1) - (b_t_1 - b_t_2)
            b_yoy_ratio = b_t / (b_t_12 + 1)
            b_ma_ratio = b_roll_mean_3 / (b_roll_mean_6 + 1)
            recent_12 = b_series[max(0, t-11):t+1]
            b_percentile_rank = (b_t >= recent_12).sum() / len(recent_12)

            # 캘린더
            quarter = (target_month - 1) // 3 + 1
            is_year_end = 1 if target_month in [11, 12, 1] else 0
            lag_weight = 1 / (1 + lag)

            # weight 기반 피처
            b_weight_features = create_conditional_weight_features(b_t, b_weight_t, b_series)

            rows.append({
                "b_t": b_t,
                "b_t_1": b_t_1,
                "b_t_12": b_t_12,
                "a_t_lag": a_t_lag,
                "max_corr": corr,
                "best_lag": float(lag),
                "month": float(target_month),

                "a_t_lag_diff": a_t_lag_diff,
                "a_momentum": a_momentum,
                "a_roll_mean_3": a_roll_mean_3,

                "b_diff": b_diff,
                "b_pct_change": b_pct_change,
                "b_yoy_growth": b_yoy_growth,
                "b_roll_mean_3": b_roll_mean_3,
                "b_roll_mean_6": b_roll_mean_6,
                "b_roll_std_3": b_roll_std_3,

                "ab_ratio": ab_ratio,
                "ab_diff": ab_diff,
                "corr_weighted_a": corr_weighted_a,

                "b_cv": b_cv,
                "b_acceleration": b_acceleration,
                "b_yoy_ratio": b_yoy_ratio,
                "b_ma_ratio": b_ma_ratio,
                "b_percentile_rank": b_percentile_rank,

                "quarter": float(quarter),
                "is_year_end": is_year_end,
                "lag_weight": lag_weight,

                "b_weight_for_high_value": b_weight_features['weight_for_high_value'],
                "b_weight_value_ratio": b_weight_features['weight_value_ratio'],
                "b_exp_weighted": b_weight_features['exp_weighted'],
                "b_weight_importance": b_weight_features['weight_importance'],

                # 타깃: log1p
                "target": np.log1p(b_t_plus_1),
            })

    return pd.DataFrame(rows)


# =============================================================================
# 5. 예측 함수 (LGBM + XGB 앙상블, expm1 역변환)
# =============================================================================

def predict_optimized(pivot, pivot_weight, pairs, model_lgb, model_xgb, months_dt):
    """32개 피처 예측 + 앙상블"""
    months = months_dt
    n_months = len(months)
    t_last = n_months - 1
    t_prev = n_months - 2
    preds = []
    target_month = months[-1].month + 1 if months[-1].month < 12 else 1

    for row in tqdm(pairs.itertuples(index=False), desc="예측"):
        leader = row.leading_item_id
        follower = row.following_item_id
        lag = int(row.best_lag)
        corr = float(row.max_corr)

        if leader not in pivot.index or follower not in pivot.index:
            continue
        if leader not in pivot_weight.index or follower not in pivot_weight.index:
            continue

        a_series = pivot.loc[leader].values.astype(float)
        b_series = pivot.loc[follower].values.astype(float)
        b_weight_series = pivot_weight.loc[follower].values.astype(float)

        if t_last - lag < 0 or t_last < 12:
            continue

        b_t = b_series[t_last]
        b_t_1 = b_series[t_prev]
        b_t_2 = b_series[t_prev - 1]
        b_t_12 = b_series[t_last - 12]
        a_t_lag = a_series[t_last - lag]
        b_weight_t = b_weight_series[t_last]

        a_t_lag_diff = a_series[t_last - lag] - a_series[t_last - lag - 1]
        a_momentum = a_series[t_last - lag] / (a_series[t_last - lag - 3] + 1)
        a_roll_mean_3 = (a_series[t_last-lag] + a_series[t_last-lag-1] + a_series[t_last-lag-2]) / 3

        b_diff = b_t - b_t_1
        b_pct_change = (b_t - b_t_1) / (b_t_1 + 1)
        b_yoy_growth = (b_t - b_t_12) / (b_t_12 + 1)
        b_roll_mean_3 = (b_series[t_last] + b_series[t_last-1] + b_series[t_last-2]) / 3
        b_roll_mean_6 = b_series[t_last-5:t_last+1].mean()
        b_roll_std_3 = np.std([b_series[t_last], b_series[t_last-1], b_series[t_last-2]])

        ab_ratio = a_t_lag / (b_t + 1)
        ab_diff = a_t_lag - b_t
        corr_weighted_a = a_t_lag * corr

        b_cv = b_roll_std_3 / (b_roll_mean_3 + 1)
        b_acceleration = (b_t - b_t_1) - (b_t_1 - b_t_2)
        b_yoy_ratio = b_t / (b_t_12 + 1)
        b_ma_ratio = b_roll_mean_3 / (b_roll_mean_6 + 1)
        recent_12 = b_series[max(0, t_last-11):t_last+1]
        b_percentile_rank = (b_t >= recent_12).sum() / len(recent_12)

        quarter = (target_month - 1) // 3 + 1
        is_year_end = 1 if target_month in [11, 12, 1] else 0
        lag_weight = 1 / (1 + lag)

        b_weight_features = create_conditional_weight_features(b_t, b_weight_t, b_series)

        X_test = np.array([[

            b_t, b_t_1, b_t_12, a_t_lag, corr, float(lag), float(target_month),

            a_t_lag_diff, a_momentum, a_roll_mean_3,

            b_diff, b_pct_change, b_yoy_growth,
            b_roll_mean_3, b_roll_mean_6, b_roll_std_3,

            ab_ratio, ab_diff, corr_weighted_a,

            b_cv, b_acceleration, b_yoy_ratio, b_ma_ratio, b_percentile_rank,

            float(quarter), is_year_end, lag_weight,

            b_weight_features['weight_for_high_value'],
            b_weight_features['weight_value_ratio'],
            b_weight_features['exp_weighted'],
            b_weight_features['weight_importance']
        ]])

        pred_lgb = model_lgb.predict(X_test)[0]
        pred_xgb = model_xgb.predict(X_test)[0]
        y_pred_log = 0.7 * pred_lgb + 0.3 * pred_xgb
        y_pred = np.expm1(y_pred_log)
        y_pred = max(0.0, float(y_pred))
        y_pred = int(round(y_pred))

        preds.append({
            "leading_item_id": leader,
            "following_item_id": follower,
            "value": y_pred,
        })

    return pd.DataFrame(preds)


# =============================================================================
# 6. 메인 실행
# =============================================================================

if __name__ == "__main__":

    print("\n" + "="*80)
    print(" DTW 기반 pairs → 2,700개 품질 필터 → 32피처 LGBM+XGB 앙상블")
    print("="*80)

    # 1) Corr + DTW 기반 pair 추출
    pairs_with_dtw = find_comovement_pairs_with_dtw(
        pivot=pivot,
        max_lag=9,
        min_nonzero=12,
        corr_threshold=0.33,
        score_threshold=40,
        use_dtw=True,
        dtw_threshold=1.0
    )

    # 2) 2,700개 목표 노이즈 제거
    pairs_for_model = remove_noise_pairs_for_2700(
        pairs=pairs_with_dtw,
        pivot=pivot,
        target_count=2700
    )

    print(f"\n최종 사용 쌍: {len(pairs_for_model)}개")

    # 3) 학습 데이터 생성
    print("\n" + "="*80)
    print("학습 데이터 생성")
    print("="*80)

    df_train_model = build_training_data_optimized(pivot, pivot_weight, pairs_for_model, months_dt)
    print(f"✓ 학습 데이터: {df_train_model.shape}")

    feature_cols = [
        'b_t', 'b_t_1', 'b_t_12', 'a_t_lag', 'max_corr', 'best_lag', 'month',
        'a_t_lag_diff', 'a_momentum', 'a_roll_mean_3',
        'b_diff', 'b_pct_change', 'b_yoy_growth', 'b_roll_mean_3', 'b_roll_mean_6', 'b_roll_std_3',
        'ab_ratio', 'ab_diff', 'corr_weighted_a',
        'b_cv', 'b_acceleration', 'b_yoy_ratio', 'b_ma_ratio', 'b_percentile_rank',
        'quarter', 'is_year_end', 'lag_weight',
        'b_weight_for_high_value', 'b_weight_value_ratio', 'b_exp_weighted', 'b_weight_importance'
    ]

    if not df_train_model.empty:
        print("\n" + "="*80)
        print("모델 학습")
        print("="*80)

        train_X = df_train_model[feature_cols].values
        train_y = df_train_model["target"].values

        print(f"학습 샘플: {len(train_X):,}, 피처: {len(feature_cols)}")

        # LGBM
        model_lgb = LGBMRegressor(
            random_state=42,
            n_estimators=700,
            learning_rate=0.01,
            max_depth=8,
            num_leaves=50,
            min_child_samples=20,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            min_split_gain=0.01
        )

        print("\nLGBM 학습 중...")
        model_lgb.fit(train_X, train_y)
        print("✓ LGBM 완료")

        # XGBoost
        model_xgb = XGBRegressor(
            random_state=42,
            n_estimators=900,
            learning_rate=0.02,
            max_depth=8,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_alpha=0.1,
            reg_lambda=0.1,
            tree_method='hist'
        )

        print("XGBoost 학습 중...")
        model_xgb.fit(train_X, train_y)
        print("✓ XGBoost 완료")

        # 4) 예측
        print("\n" + "="*80)
        print("예측")
        print("="*80)

        submission = predict_optimized(
            pivot, pivot_weight,
            pairs_for_model,
            model_lgb, model_xgb,
            months_dt
        )
        print(f"\n✓ 예측 완료: {len(submission)}개")

        os.makedirs("_result", exist_ok=True)
        date_str = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        out_path = f"_result/jh_2700dtw_{date_str}.csv"
        submission.to_csv(out_path, index=False)

        print("\n 최종 완료!")
        print(f"  - 저장 위치: {out_path}")
        print("  - Stage1: Corr+DTW pairs → 2,700개 품질 필터")
        print("  - Stage2: 32개 피처 + log1p target")
        print("  - Stage3: LGBM + XGB 앙상블 (0.7 / 0.3)")
        print("="*80)
    else:
        print("학습 데이터가 비어 있습니다. 파라미터나 필터 조건을 확인하세요.")

데이터 로드 및 월별 집계
데이터 로드 완료: 100개 품목, 43개월

 DTW 기반 pairs → 2,700개 품질 필터 → 32피처 LGBM+XGB 앙상블


Finding pairs (DTW): 100%|███████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.16it/s]



Pairs with DTW: 3368개
  leading_item_id following_item_id  best_lag  max_corr   p_value  \
0        AANGBULD          ZCELVYQU         7  0.677701  0.000006   
1        AANGBULD          ZKENOUDA         1  0.599969  0.000027   
2        AANGBULD          DEWLVASR         6  0.640221  0.000020   
3        AANGBULD          NAQIHUKZ         2  0.525490  0.000419   
4        AANGBULD          GKQIJYDH         6  0.582501  0.000155   

   dtw_distance  comovement_score  
0      0.304327        120.909908  
1      0.513452        119.569775  
2      0.490232        114.993153  
3      0.471456        112.170840  
4      0.464203        109.014231  

2,700개 목표 노이즈 제거 (중간 모드)
원본: 3,368개
목표: 2,700개 (약 80% 유지)

품질 평가 중...


평가: 100%|███████████████████████████████████████████████████████████████████████| 3368/3368 [00:04<00:00, 721.77it/s]



노이즈 제거 완료!
원본: 3,368개
제거: 611개 (18.1%)
유지: 2,757개 (81.9%)
임계값: 55점

품질 점수 분포:
  - 평균: 65.5점
  - 중앙값: 67.0점
  - 최소: 36.0점
  - 최대: 90.0점

선택된 쌍의 점수:
  - 평균: 69.2점
  - 최소: 55.0점

점수 구간별 분포:
    0- 30점: 전체    0개, 선택    0개
   30- 40점: 전체   29개, 선택    0개
   40- 50점: 전체  205개, 선택    0개
   50- 60점: 전체  708개, 선택  331개
   60- 70점: 전체 1,002개, 선택 1,002개
   70- 80점: 전체 1,212개, 선택 1,212개
   80- 90점: 전체  208개, 선택  208개
   90-100점: 전체    4개, 선택    4개

기준별 평균 점수 (선택된 쌍):
  0 비율           : 평균   9.8점 (최대 10점)
  변동계수           : 평균   7.8점 (최대 10점)
  스파이크           : 평균   6.2점 (최대 10점)
  안정성            : 평균   5.0점 (최대 15점)
  트렌드 일치         : 평균   3.9점 (최대 10점)
  방향 일치도         : 평균  10.0점 (최대 10점)
  통계적 유의성        : 평균   7.7점 (최대 10점)
  자기상관           : 평균   7.8점 (최대 10점)
  이상치 비율         : 평균   8.7점 (최대 10점)
  정규성            : 평균   2.2점 (최대 5점)

최종 사용 쌍: 2757개

학습 데이터 생성
✓ 학습 데이터: (82710, 32)

모델 학습
학습 샘플: 82,710, 피처: 31

LGBM 학습 중...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhea

예측: 2757it [00:14, 191.36it/s]


✓ 예측 완료: 2757개

 최종 완료!
  - 저장 위치: _result/jh_2700dtw_20251128_163251.csv
  - Stage1: Corr+DTW pairs → 2,700개 품질 필터
  - Stage2: 32개 피처 + log1p target
  - Stage3: LGBM + XGB 앙상블 (0.7 / 0.3)
